# Nome

In [ ]:
nome_da_dupla = 'Luiza Higino & Thiago Martins'
print(f'Este trabalho final foi desenvolvido por {nome_da_dupla}')

Este trabalho final foi desenvolvido por Luiza Higino & Thiago Martins


# Install

In [ ]:
%%capture
! pip install transformers
! pip install datasets
! pip install openai
! pip install ipdb
! pip install unidecode

# Import

In [ ]:
import json
import requests
import numpy as np
from pprint import pprint
from datasets import load_dataset
from random import randrange
import os
import openai
import random
from tqdm import tqdm
import torch
from collections import Counter
from unidecode import unidecode

# Load Dataset

In [ ]:
train_dataset = load_dataset("AmazonScience/massive", "pt-PT", split='train')

# pprint(dataset.info.__dict__)
print('Example:', train_dataset[0])

Example: {'id': '1', 'locale': 'pt-PT', 'partition': 'train', 'scenario': 16, 'intent': 48, 'utt': 'acorda-me às nove da manhã na sexta-feira', 'annot_utt': 'acorda-me às [time : nove da manhã] na [date : sexta-feira]', 'worker_id': '14', 'slot_method': {'slot': ['time', 'date'], 'method': ['localization', 'translation']}, 'judgments': {'worker_id': ['6', '8', '12'], 'intent_score': [1, 1, 1], 'slots_score': [1, 1, 2], 'grammar_score': [4, 4, 4], 'spelling_score': [1, 1, 2], 'language_identification': ['target', 'target', 'target']}}


In [ ]:
valid_dataset = load_dataset("AmazonScience/massive", "pt-PT", split='validation')
test_dataset = load_dataset("AmazonScience/massive", "pt-PT", split='test')

print('\n\nDataset de treino:', len(train_dataset))
print('Dataset de validação:', len(valid_dataset))
print('Dataset de teste:', len(test_dataset))



Dataset de treino: 11514
Dataset de validação: 2033
Dataset de teste: 2974


In [ ]:
print(train_dataset.info.features['scenario'])
(train_dataset.info.features['scenario'].int2str(12))

ClassLabel(names=['social', 'transport', 'calendar', 'play', 'news', 'datetime', 'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather'], id=None)


'qa'

# Tradução para português

In [ ]:
en_scenarios =   ['social', 'transport', 'calendar', 'play', 'news', 'datetime', 
               'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 
                'cooking', 'takeaway', 'music', 'alarm', 'weather']

pt_scenarios = ['social', 'transporte', 'agenda', 'tocar/jogar', 'notícias', 'horaedata', 
                'recomendação', 'email', 'iot', 'geral', 'áudio', 'listas', 'perguntaserespostas', 
                'culinária', 'retirada', 'musica', 'alarme', 'tempo']

# Funções

## Prompt

In [ ]:
def amostra_exemplos(lista_cenarios, num_exemplos, seed=11):
  """
  Amostra aleatoriamente exemplos de uma lista de cenários
  """
  random.seed(seed)
  np.random.seed(seed)
  indices = np.random.randint(len(lista_cenarios), size=num_exemplos)
  final_list = [lista_cenarios[idx] for idx in indices]
  return final_list

def cria_lista_dos_cenarios(dataset, lista_cenarios):
  """
  Cria um dicionário com as listas dos cenários especificados
  """
  # Inicializa um dicionário que vai armazenar as listas
  top_dict = {}
  for key in lista_cenarios:
    top_dict[key] = []

  # Percorre o dataset e salva os exemplos dos cenários especificados
  for idx in range(len(dataset)):
    comando = train_dataset[idx]['utt']
    scenario = en_scenarios[train_dataset[idx]['scenario']]
    if scenario in top_dict.keys():
      top_dict[scenario].append(comando)

  return top_dict

In [ ]:
listas_cenarios = cria_lista_dos_cenarios(train_dataset, lista_cenarios=en_scenarios)

In [ ]:
listas_cenarios.keys()

dict_keys(['social', 'transport', 'calendar', 'play', 'news', 'datetime', 'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather'])

In [ ]:
len(listas_cenarios['news'])

503

In [ ]:
amostra_exemplos(listas_cenarios['calendar'], 5)

['encontrar o meu próximo evento e apagá-lo',
 'lembrar-me disto amanhã',
 'quando é o meu aniversário',
 'qual vai ser a duração do evento',
 'existem muitos eventos importantes neste mês anote isso']

In [ ]:
prompt_inicial = 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinária, retirada, musica, alarme, tempo.\n\n'

def new_prompt_scenarios(dataset, pergunta_idx: int, examples_in_prompt=10, seed=11):
  """
  Cria um prompt aleatório, com N exemplos.
  O número de exemplos deve ser menor do que o número de cenários,
  pois esta função não coloca cenários repetidos no mesmo prompt.
  Desta forma a diversidade de cenários é garantida.
  A pergunta é fixa, e o índice correspondente é recebido por parâmetro

  Parameters
  ----------  
  dataset: 
    dataset onde os exemplos e pergunta serão buscados

  pergunta_idx: int
    índice da pergunta no dataset

  examples_in_prompt: int, default = 10
    número de exemplos que serão inseridos no prompt

  """
  random.seed(seed)
  scenarios_eval = {}
  cenarios_selecionados = []
  exemplos_selecionados = []

  # Início do Prompt
  prompt = prompt_inicial
  target = ''

  # Cria um conjunto de cenários únicos
  while len(cenarios_selecionados) < examples_in_prompt:
    random_idx = randrange(len(dataset))
    exemplo = dataset[random_idx]
    cenario = pt_scenarios[exemplo['scenario']]
    if cenario not in cenarios_selecionados:
        cenarios_selecionados.append(cenario)
        exemplos_selecionados.append(exemplo['utt'])

  # Monta o prompt com os exemplos selecionados
  for i in range(len(exemplos_selecionados)):
    text = exemplos_selecionados[i]
    cenario = cenarios_selecionados[i]   
    prompt = prompt + f'Exemplo {i+1}\nComando: {text}\nCenário: {cenario}\n\n'   
    
  return prompt

def prompt_cenario_unico(dataset, num_exemplos, cenario, cenatio_pt, seed=11):
  """
  Cria um prompt de N exemplos, sendo todos do cenário especificado
  """
  random.seed(seed)
  exemplos_selecionados = amostra_exemplos(listas_cenarios[cenario], num_exemplos, seed)
  prompt = prompt_inicial
  # Monta o prompt com os exemplos selecionados
  for i in range(num_exemplos):
    text = exemplos_selecionados[i]
    cenario = cenatio_pt
    prompt = prompt + f'Exemplo {i+1}\nComando: {text}\nCenário: {cenario}\n\n'


  return prompt

def prob_prompt_scenarios(dataset, examples_in_prompt=10, seed=11):
  """
  Cria um prompt aleatório, com N exemplos.
  O número de exemplos deve ser menor do que o número de cenários,
  pois esta função não coloca cenários repetidos no mesmo prompt.
  Desta forma a diversidade de cenários é garantida.
  A pergunta é fixa, e o índice correspondente é recebido por parâmetro

  Parameters
  ----------  
  dataset: 
    dataset onde os exemplos e pergunta serão buscados

  pergunta_idx: int
    índice da pergunta no dataset

  examples_in_prompt: int, default = 10
    número de exemplos que serão inseridos no prompt

  """
  random.seed(seed)
  scenarios_eval = {}
  cenarios_selecionados = []
  exemplos_selecionados = []

  # Início do Prompt
  prompt = 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo\n\n'
  target = ''

  # Cria um conjunto de cenários únicos
  while len(cenarios_selecionados) < examples_in_prompt:
    random_idx = randrange(len(dataset))
    exemplo = dataset[random_idx]
    cenario = pt_scenarios[exemplo['scenario']]
    cenarios_selecionados.append(cenario)
    exemplos_selecionados.append(exemplo['utt'])

  # Monta o prompt com os exemplos selecionados
  for i in range(len(exemplos_selecionados)):
    text = exemplos_selecionados[i]
    cenario = cenarios_selecionados[i]   
    prompt = prompt + f'Exemplo {i+1}\nComando: {text}\nCenário: {cenario}\n\n'   
    
  return prompt


def prompt_fixo():
  prompt = 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinária, retirada, musica, alarme, tempo\n\n'

  #prompt += 'Exemplo 1\n \
  #           Comando: por favor remove a lista de mantimentos do meu conjunto de listas \nCenário: listas\n\n'
  prompt += 'Exemplo 1\n \
             Comando: quero ouvir algumas músicas \nCenário: tocar/jogar\n'
  prompt += 'Exemplo 2\n \
             Comando: abre o rádio \nCenário: tocar/jogar\n'
  prompt += 'Exemplo 3\n \
             Comando: toca todas as minhas músicas latinas favoritas\nCenário: tocar/jogar\n\n'

  """             
  prompt += 'Exemplo 5\n \
             Comando:  diz me os detalhes do tempo para dia quinze de março\nCenário: tempo\n\n'  
  prompt += 'Exemplo 6\n \
             Comando:  qual a temperatura para a semana toda \nCenário: tempo\n\n'
  prompt += 'Exemplo 7\n \
             Comando:  há possibilidade de haver neve quarta-feira \nCenário: tempo\n\n'
  
  prompt += 'Exemplo 8\n \
             Comando:  lembrar de enviar e-mail para a minha irmã \nCenário: agenda\n\n'
  prompt += 'Exemplo 9\n \
             Comando:  há alguma coisa importante para hoje \nCenário: agenda\n\n'
  prompt += 'Exemplo 10\n \
             Comando:  adicionar evento ao calendário \nCenário: agenda\n\n'
  
  prompt += 'Exemplo 8\n \
             Comando:  reenviar um email à susana \nCenário: email\n\n'
  prompt += 'Exemplo 9\n \
             Comando:  vamos responder a este email \nCenário: email\n\n'
  prompt += 'Exemplo 10\n \
             Comando:  qual foi o último email que eu recebi \nCenário: email\n\n'
  """

  return prompt
             

In [ ]:
top =[('calendar', 280),
 ('play', 260),
 ('qa', 214),
 ('email', 157),
 ('weather', 126),
 ('general', 122),
 ('iot', 118),
 ('lists', 112),
 ('transport', 110)]

top_key = [item[0] for item in top]

In [ ]:
a = new_prompt_scenarios(train_dataset, pergunta_idx = 5, seed=11)
print(a)

Defina o cenário para o último exemplo abaixo.

Cenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinária, retirada, musica, alarme, tempo.

Exemplo 1
Comando: por favor remove a lista de mantimentos do meu conjunto de listas
Cenário: listas

Exemplo 2
Comando: que idade tem o marco paulo
Cenário: perguntaserespostas

Exemplo 3
Comando: põe o podcast tubo de ensaio
Cenário: tocar/jogar

Exemplo 4
Comando: compra um bilhete de comboio de lisboa para faro
Cenário: transporte

Exemplo 5
Comando: que horas são em lisboa
Cenário: horaedata

Exemplo 6
Comando: títulos recentes
Cenário: notícias

Exemplo 7
Comando: listar desportos universitários para amanhã em lisboa
Cenário: recomendação

Exemplo 8
Comando: registar reclamação no twitter
Cenário: social

Exemplo 9
Comando: mudar a cor das minhas luzes para modo aleatório
Cenário: iot

Exemplo 10
Comando: quero ver cada um dos meus lembretes pendente

In [ ]:
b = prob_prompt_scenarios(train_dataset, seed=55)
print(b)

Defina o cenário para o último exemplo abaixo.

Cenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo

Exemplo 1
Comando: coloca folk como favorito
Cenário: tocar/jogar

Exemplo 2
Comando: o alarme programado para as nove horas da noite de quinta-feira
Cenário: alarme

Exemplo 3
Comando: encontre-me as notícias mais populares do dia
Cenário: notícias

Exemplo 4
Comando: que reuniões estão disponíveis em março
Cenário: agenda

Exemplo 5
Comando: diminui volume
Cenário: áudio

Exemplo 6
Comando: em que dia vai estar ensolarado
Cenário: tempo

Exemplo 7
Comando: enumerar todos os eventos do meu calendário de março
Cenário: agenda

Exemplo 8
Comando: eu gosto de qualquer coisa do tiago nacarato
Cenário: musica

Exemplo 9
Comando: há um cozinheiro de chili este fim-de-semana
Cenário: agenda

Exemplo 10
Comando: escrever email para meu colega de trabalho para subm

In [ ]:
c = prompt_cenario_unico(train_dataset, 5, 'calendar', 'agenda')
print(c)

Defina o cenário para o último exemplo abaixo.

Cenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinária, retirada, musica, alarme, tempo.

Exemplo 1
Comando: encontrar o meu próximo evento e apagá-lo
Cenário: agenda

Exemplo 2
Comando: lembrar-me disto amanhã
Cenário: agenda

Exemplo 3
Comando: quando é o meu aniversário
Cenário: agenda

Exemplo 4
Comando: qual vai ser a duração do evento
Cenário: agenda

Exemplo 5
Comando: existem muitos eventos importantes neste mês anote isso
Cenário: agenda




## Query e Get

In [ ]:
def hf_query(payload, model_id):
  """
  Query (post) utilizando a API da HuggingFace
  Utilizado para enviar uma requisição e receber uma resposta
  """
  api_token = ""
  headers = {"Authorization": f"Bearer {api_token}"}
  API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
  response = requests.post(API_URL, headers=headers, json=payload)
  return response.json()
 
def hf_get(func):
  """
  Método get para a API da HuggingFace
  Utilizado para comandos que não precisa enviar um payload
  """
  api_token = ""
  headers = {"Authorization": f"Bearer {api_token}"}
  API_URL = 'https://api-inference.huggingface.co/'+func
  print(API_URL)
  response = requests.get(API_URL, headers=headers)
  return response.json()

## Evaluate

In [ ]:
def evaluate_gpt_dataset (valid_dataset, predicted):
  """ -------- não testei ainda
  Recebe um dataset de validação e os valores previstos
  Contam quantos acertos foram obtidos, retorna a acurácia
  """

  acertou = 0
  for idx, data in enumerate(valid_dataset):
      target = str(d['generated_text']).rsplit(':', 1)[-1].replace(' ', '')   
      if target == predicted[idx]:
        acertou += 1
  
  acc = acertou / len(predicted)
  return acc

def evaluate_bert_dataset (valid_dataset, predicted):
  """
  Recebe um dataset de validação e os valores previstos
  Contam quantos acertos foram obtidos, retorna a acurácia
  """

  acertou = 0
  for idx, data in enumerate(valid_dataset):
      target = pt_scenarios[valid_dataset['scenario']]
      if target == predicted[idx]:
        acertou += 1
  
  acc = acertou / len(predicted)
  return acc

def evaluate_lists(target, predicted):
  """
  Verifica a acurácia do moldeo
  Ignora acentos, maiúsculas e minúsculas
  """
  acertos = 0
  for idx in range(len(target)):

    if unidecode(target[idx].lower()) == unidecode(predicted[idx].lower()):
      acertos += 1
    
  acc = acertos / len(target)
  return acc

## Valida Modelo

In [ ]:
def adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt, mask=True):
  """
  Adiciona uma pergunta no final do prompt
  """
  text = dataset[pergunta_idx]['utt']
  cenario = pt_scenarios[dataset[pergunta_idx]['scenario']]
  prompt = prompt + f'Exemplo {examples_in_prompt+1}\nComando: {text}\nCenário:' 
  target = cenario

  if mask: # Modelos tipo BERT precisam da máscara
    prompt += ' [MASK]' 

  return prompt, target

def valida_modelo(model_id, dataset, dict_key='generated_text', num_exemplos=200, mask=False, seed=11):
  """
  Testa um modelo no dataset especificado, com o número de exemplos especificado
  """
  predicted = []
  target_list = []

  if num_exemplos == -1:
    num_exemplos = len(dataset)
    
  for pergunta_idx in tqdm(range(num_exemplos)):
    prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx, seed=seed)
    entrada, target = adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt=10, mask=mask)
    try:
      data = hf_query(entrada, model_id)[0] # faz a requisição pro modelo
    except:
      data = hf_query(entrada, model_id)
    
    try:
      if dict_key == 'generated_text': # GPT
        pred = str(data[dict_key]).rsplit(':', 1)[-1].replace(' ', '')  
      else: # Modelos BERT
        pred =data['token_str']

      # Salva uma lista de predições e de groundtruth
      predicted.append(pred)
      target_list.append(target)
    except:
      print('Erro:', data)
      predicted.append('erro')
      target_list.append(target)
  
  acc = evaluate_lists(target_list, predicted)
  return acc, predicted

def valida_modelo_prob(model_id, dataset, dict_key='generated_text', num_exemplos=200, mask=False, seed=11):
  """
  Testa um modelo no dataset especificado, com o número de exemplos especificado
  """
  predicted = []
  target_list = []

  if num_exemplos == -1:
    num_exemplos = len(dataset)
    
  for pergunta_idx in tqdm(range(num_exemplos)):
    prompt = prob_prompt_scenarios(train_dataset, seed=seed)
    entrada, target = adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt=10, mask=mask)
    try:
      data = hf_query(entrada, model_id)[0] # faz a requisição pro modelo
    except:
      data = hf_query(entrada, model_id)
    
    try:
      if dict_key == 'generated_text': # GPT
        pred = str(data[dict_key]).rsplit(':', 1)[-1].replace(' ', '')  
      else: # Modelos BERT
        pred =data['token_str']

      # Salva uma lista de predições e de groundtruth
      predicted.append(pred)
      target_list.append(target)
    except:
      print('Erro:', data)
      predicted.append('erro')
      target_list.append(target)
  
  acc = evaluate_lists(target_list, predicted)
  return acc, predicted


def valida_modelo_list(model_id, dataset, dict_key='generated_text', num_exemplos=200, mask=False):
  """
  Testa um modelo no dataset especificado, com o número de exemplos especificado
  --- nessa versão ele faz faz apenas uma requisição, com todas os prompts em uma lista
  """
  predicted = []
  target_list = []
  entrada_list = []

  for pergunta_idx in range(num_exemplos):
    prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
    entrada, target = adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt=10, mask=mask)
    entrada_list.append(entrada)

  #import ipdb; ipdb.set_trace()
  data_list = hf_query(entrada_list, model_id)  
  
  for data in data_list:
    if dict_key == 'generated_text': # GPT
      pred = str(data[dict_key]).rsplit(':', 1)[-1].replace(' ', '')  
    else: # Modelos BERT
      pred = data['token_str']

    # Salva uma lista de predições e de groundtruth
    predicted.append(pred)
    target_list.append(target)
  
  acc = evaluate_lists(target_list, predicted)
  return acc, predicted

def valida_modelo_prompt_fixo(model_id, dataset, prompt, dict_key='generated_text', num_exemplos=200, mask=False):
  """
  Testa um modelo no dataset especificado, com o número de exemplos especificado
  """
  predicted = []
  target_list = []

  if num_exemplos == -1:
    num_exemplos = len(dataset)
    
  for pergunta_idx in tqdm(range(num_exemplos)):    
    entrada, target = adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt=10, mask=mask)
    try:
      data = hf_query(entrada, model_id)[0] # faz a requisição pro modelo
    except:
      data = hf_query(entrada, model_id)
    
    try:
      if dict_key == 'generated_text': # GPT
        pred = str(data[dict_key]).rsplit(':', 1)[-1].replace(' ', '')  
      else: # Modelos BERT
        pred = data['token_str']

      # Salva uma lista de predições e de groundtruth
      predicted.append(pred)
      target_list.append(target)
    except:
      print('Erro:', data)
      predicted.append('erro')
      target_list.append(target)
  
  acc = evaluate_lists(target_list, predicted)
  return acc, predicted

In [ ]:
pergunta_idx = 3
model_id = 'EleutherAI/gpt-j-6B'

prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)
data = hf_query(entrada, model_id)

print (data)

[{'generated_text': 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinária, retirada, musica, alarme, tempo.\n\nExemplo 1\nComando: por favor remove a lista de mantimentos do meu conjunto de listas\nCenário: listas\n\nExemplo 2\nComando: que idade tem o marco paulo\nCenário: perguntaserespostas\n\nExemplo 3\nComando: põe o podcast tubo de ensaio\nCenário: tocar/jogar\n\nExemplo 4\nComando: compra um bilhete de comboio de lisboa para faro\nCenário: transporte\n\nExemplo 5\nComando: que horas são em lisboa\nCenário: horaedata\n\nExemplo 6\nComando: títulos recentes\nCenário: notícias\n\nExemplo 7\nComando: listar desportos universitários para amanhã em lisboa\nCenário: recomendação\n\nExemplo 8\nComando: registar reclamação no twitter\nCenário: social\n\nExemplo 9\nComando: mudar a cor das minhas luzes para modo aleatório\nCenário: iot\n\nExem

## Verifica respostas

In [ ]:
def verifica_respostas (dataset, pred):
  scenarios = []
  for idx in range(len(pred)):
    scenarios.append(pt_scenarios[dataset[idx]['scenario']])

  counter_gabarito = Counter(scenarios)
  counter_model = Counter(pred)
  # counter_GPTJ = Counter(pred_GPTJ)

  dict_result = {}
  for key in counter_gabarito.keys():
    dict_result[key] = []
  dict_counter = {}

  for i, scenario in enumerate(scenarios):
    dict_result[scenario].append(pred[i])

  for key in dict_result.keys():
    c = Counter(dict_result[key])
    dict_counter[key] = c.most_common(5)

  return dict_counter


# Modelos

## XGLM

In [ ]:
model_id = 'facebook/xglm-564M'
prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)
data = hf_query(entrada, model_id) # faz a requisição pro modelo
print(data)

{'error': 'Model facebook/xglm-564M is currently loading', 'estimated_time': 45.162384033203125}


In [ ]:
acc_XGLM = valida_modelo('facebook/xglm-564M', dataset=valid_dataset)
print('\nAcurácia:', acc_XGLM)

## BLOOM

In [ ]:
model_id = 'bigscience/bloom-560m'
prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)
data = hf_query(entrada, model_id) # faz a requisição pro modelo
print(data)

[{'generated_text': 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, hora e data, recomendacao, email, iot, geral, audio, listas, perguntas e respostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: por favor remove a lista de mantimentos do meu conjunto de listas\nCenário: listas\n\nExemplo 2\nComando: que idade tem o marco paulo\nCenário: perguntas e respostas\n\nExemplo 3\nComando: põe o podcast tubo de ensaio\nCenário: play\n\nExemplo 4\nComando: compra um bilhete de comboio de lisboa para faro\nCenário: transporte\n\nExemplo 5\nComando: que horas são em lisboa\nCenário: hora e data\n\nExemplo 6\nComando: títulos recentes\nCenário: notícias\n\nExemplo 7\nComando: listar desportos universitários para amanhã em lisboa\nCenário: recomendação\n\nExemplo 8\nComando: registar reclamação no twitter\nCenário: social\n\nExemplo 9\nComando: mudar a cor das minhas luzes para modo aleatório\nCenário: iot\n\nExem

In [ ]:
acc_BLOOM = valida_modelo('bigscience/bloom-560m', dataset=valid_dataset)
print('\nAcurácia:', acc_BLOOM)

100%|██████████| 200/200 [00:32<00:00,  6.14it/s]


Acurácia: (0.15, ['música', 'música', 'música', 'lista', 'música', 'recomendações', 'recomendações', 'agenda', 'música', 'música', 'lista', 'notícias', 'hor', 'música', 'notícias', 'música', 'música', 'áudio', 'horário', 'música', 'música', 'música', 'notícias', 'horário', 'música', 'notícias', 'horário', 'horário', 'horário', 'horário', 'notícias', 'música', 'horário', 'notícias', 'música', 'horário', 'horário', 'notícias', 'música', 'temperatura', 'recomendações', 'horário', 'agenda', 'horário', 'horário', 'notícias', 'i', 'recomendações', 'música', 'música', 'horário', 'horário', 'horário', 'recomendações', 'horário', 'recomendações', 'horário', 'horário', 'al', 'lista', 'recomendações', 'música', 'agenda', 'notícias', 'horário', 'al', 'horário', 'música', 'música', 'horário', 'notícias', 'música', 'horário', 'horário', 'música', 'horário', 'música', 'música', 'horário', 'horário', 'horário', 'horário', 'notícias', 'música', 'áudio', 'recomendações', 'música', 'música', 'música', '

In [ ]:
acc_BLOOM = valida_modelo_prob('bigscience/bloom-560m', dataset=valid_dataset)
print('\nAcurácia:', acc_BLOOM)

100%|██████████| 200/200 [04:23<00:00,  1.32s/it]


Acurácia: (0.055, ['recomendações', 'recomendações', 'lista', 'lista', 'recomendações', 'recomendações', 'recomendações', 'recomendações', 'música', 'música', 'lista', 'lista', 'previsão', 'listas', 'listas', 'listas', 'listas', 'recomendações', 'horário', 'música', 'música', 'música', 'lista', 'al', 'recomendações', 'lista', 'horário', 'horário', 'al', 'al', 'lista', 'lista', 'horário', 'notícias', 'música', 'agenda', 'recomendações', 'recomendações', 'recomendações', 'temperatura', 'recomendações', 'horário', 'agenda', 'horário', 'lista', 'recomendações', 'recomendações', 'listas', 'listas', 'recomendações', 'horário', 'horário', 'lista', 'recomendações', 'lista', 'listas', 'horário', 'recomendações', 'al', 'lista', 'recomendações', 'listas', 'recomendações', 'notícias', 'lista', 'al', 'al', 'recomendações', 'recomendações', 'recomendações', 'recomendações', 'recomendações', 'al', 'agenda', 'listas', 'agenda', 'lista', 'lista', 'horário', 'horário', 'horário', 'lista', 'recomendaçõe

## GPT-J

In [ ]:
model_id = 'EleutherAI/gpt-j-6B'
prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)
data = hf_query(entrada, model_id) # faz a requisição pro modelo
print(data[0])

{'generated_text': 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, hora e data, recomendacao, email, iot, geral, audio, listas, perguntas e respostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: por favor remove a lista de mantimentos do meu conjunto de listas\nCenário: listas\n\nExemplo 2\nComando: que idade tem o marco paulo\nCenário: perguntas e respostas\n\nExemplo 3\nComando: põe o podcast tubo de ensaio\nCenário: play\n\nExemplo 4\nComando: compra um bilhete de comboio de lisboa para faro\nCenário: transporte\n\nExemplo 5\nComando: que horas são em lisboa\nCenário: hora e data\n\nExemplo 6\nComando: títulos recentes\nCenário: notícias\n\nExemplo 7\nComando: listar desportos universitários para amanhã em lisboa\nCenário: recomendação\n\nExemplo 8\nComando: registar reclamação no twitter\nCenário: social\n\nExemplo 9\nComando: mudar a cor das minhas luzes para modo aleatório\nCenário: iot\n\nExemp

In [ ]:
acuracia_GPTJ, pred_GPTJ = valida_modelo('EleutherAI/gpt-j-6B', dataset=valid_dataset, num_exemplos=200, seed=14)
print('\nAcurácia:', acuracia_GPTJ)

  4%|▍         | 8/200 [00:28<27:07,  8.48s/it]

Erro: [{'generated_text': 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendacao, email, iot, geral, audio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo.\n\nExemplo 1\nComando: checa os recentes artigos de notícias da sic\nCenário: notícias\n\nExemplo 2\nComando: emma stone é solteira\nCenário: perguntaserespostas\n\nExemplo 3\nComando: quais emails são novos\nCenário: email\n\nExemplo 4\nComando: reserva um bilhete de comboio para a covilhã\nCenário: transporte\n\nExemplo 5\nComando: o que é que eu reproduzi mais vezes\nCenário: música\n\nExemplo 6\nComando: o que deveria fazer hoje\nCenário: geral\n\nExemplo 7\nComando: por favor convida o marcos e o giuliano para uma reunião na quinta feira às três da tarde\nCenário: calendário\n\nExemplo 8\nComando: quero ouvir música country\nCenário: tocar/jogar\n\nExemplo 9\nComando: remove a minha lista de mercearias\nCenário: listas\n\nE

100%|██████████| 200/200 [01:51<00:00,  1.79it/s]


Acurácia: 0.02


In [ ]:
acuracia_GPTJ, pred_GPTJ = valida_modelo_prob('EleutherAI/gpt-j-6B', dataset=valid_dataset, num_exemplos=200, seed=14)
print('\nAcurácia:', acuracia_GPTJ)

100%|██████████| 200/200 [02:23<00:00,  1.39it/s]


Acurácia: 0.02


## FLAN-T5

## Prompt variado

In [ ]:
prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)

model_id = 'google/flan-t5-large'
data = hf_query(entrada, model_id) # faz a requisição pro modelo
#print(entrada)
print(data)

[{'generated_text': 'tempo'}]


In [ ]:
#tocarjogar
acuracia_FLANT5, pred_FLANT5 = valida_modelo(model_id = 'google/flan-t5-large', dataset=valid_dataset, num_exemplos=10, seed=11)
print('\nAcurácia:', acuracia_FLANT5)

100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


Acurácia: 0.1


## UL2-20B


In [ ]:
pergunta_idx = 11
prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10, mask=False)

In [ ]:
model_id = 'google/ul2'
#entrada = exemplo['prompt']  # GPT-J não usa máscara
data = hf_query(entrada, model_id) # faz a requisição pro modelo

print(data)

In [ ]:
acuracia_UL2 = valida_modelo('google/ul2', dataset=valid_dataset)
print('\nAcurácia:', acuracia_UL2)

## Teste - DistilBert

In [ ]:
model_id = "distilbert-base-uncased"
 # get yours at hf.co/settings/tokens

In [ ]:
acuracia_DistilBert = valida_modelo("distilbert-base-uncased", dataset=valid_dataset, mask=True, dict_key='token_str')
print('\nAcurácia:', acuracia_DistilBert)

## Teste Bertimbau

In [ ]:
%%capture
! pip install ipdb

In [ ]:
model_id = 'neuralmind/bert-base-portuguese-cased'
predicted = []
for pergunta_idx in range(len(valid_dataset)):
  
  prompt = new_prompt_scenarios(train_dataset, pergunta_idx=pergunta_idx)
  entrada, target = adiciona_pergunta(prompt, valid_dataset, pergunta_idx, examples_in_prompt=10)  
  #import ipdb; ipdb.set_trace()
  #print(entrada)
  data = hf_query(entrada, model_id)[0] # faz a requisição pro modelo
  
  # Fica em loop até conseguir
  while True:
    try:
      predicted.append(data['token_str'])
      break
    except:  
      print(data)

  if pergunta_idx % 20 == 0:
    print(predicted)

In [ ]:
acuracia_BERTimbau = valida_modelo('neuralmind/bert-base-portuguese-cased', dataset=valid_dataset, mask=True, dict_key='token_str')
print('\nAcurácia:', acuracia_BERTimbau)

# API OpenAI

In [ ]:
openai.api_key = ''

In [ ]:
def openai_query (model_id, prompt):
  response = openai.Completion.create(
    model = model_id,
    prompt = prompt,
    temperature = 1,
    max_tokens = 256,
    top_p = 1,
    frequency_penalty = 0,
    presence_penalty = 0
  )
  return response

In [ ]:
import time

In [ ]:
def valida_davinci(model_id, dataset, prompt, dict_key='generated_text', num_exemplos=200, mask=False):
  """
  Testa um modelo no dataset especificado, com o número de exemplos especificado
  """
  predicted = []
  target_list = []

  if num_exemplos == -1:
    num_exemplos = len(dataset)

  for pergunta_idx in tqdm(range(num_exemplos)):
    time.sleep(6)
    entrada, target = adiciona_pergunta(prompt, dataset, pergunta_idx, examples_in_prompt=10, mask=mask)
    data = openai_query(model_id, entrada) # faz a requisição pro modelo    
    pred = data['choices'][0]['text']
    pred = pred.replace(' ','')
    # Salva uma lista de predições e de groundtruth
    predicted.append(pred)
    target_list.append(target)
  
  acc = evaluate_lists(target_list, predicted)
  return acc, predicted

# Loop de teste dos modelos

In [ ]:
## Loop para modelos exceto davinci.

prompts = [prob_prompt_scenarios(train_dataset, seed=55)]

print(prompts)

acc = []
pred = []
model_id = [['google/flan-t5-large', False, None],
            ['neuralmind/bert-base-portuguese-cased', True, 'token_str'],
            ['distilbert-base-uncased', True, 'token_str'],
            ['EleutherAI/gpt-j-6B', False, None]]


for model in model_id:
  for prompt in prompts: 
    # print(prompt)
    accuracy, prediction = 0, 0

    if model[2] != None:
      print( 'entrou')
      accuracy, prediction = valida_modelo_prompt_fixo(model_id=model[0], dataset=valid_dataset, prompt=prompt,  dict_key = model[2], num_exemplos=-1, mask = model[1])
    else:
      accuracy, prediction = valida_modelo_prompt_fixo(model_id=model[0], dataset=valid_dataset, prompt=prompt, num_exemplos=-1)

    print(f'\nModel = {model} || Acurácia: {accuracy}')

    acc.append(accuracy)
    pred.append(prediction)

for i, p in enumerate(pred):
  print(prompt[0])
  print(f'\nAcurácia prompt {i+1}: {acc[i]*100: .2f}%')
  print(verifica_respostas(valid_dataset, p))


['Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: coloca folk como favorito\nCenário: tocar/jogar\n\nExemplo 2\nComando: o alarme programado para as nove horas da noite de quinta-feira\nCenário: alarme\n\nExemplo 3\nComando: encontre-me as notícias mais populares do dia\nCenário: notícias\n\nExemplo 4\nComando: que reuniões estão disponíveis em março\nCenário: agenda\n\nExemplo 5\nComando: diminui volume\nCenário: áudio\n\nExemplo 6\nComando: em que dia vai estar ensolarado\nCenário: tempo\n\nExemplo 7\nComando: enumerar todos os eventos do meu calendário de março\nCenário: agenda\n\nExemplo 8\nComando: eu gosto de qualquer coisa do tiago nacarato\nCenário: musica\n\nExemplo 9\nComando: há um cozinheiro de chili este fim-de-semana\nCenário: agenda\n\nExemplo 10\nComando: escrever 



  0%|          | 0/2033 [00:00<?, ?it/s]

  0%|          | 1/2033 [00:01<58:43,  1.73s/it]

  0%|          | 2/2033 [00:03<56:57,  1.68s/it]

  0%|          | 3/2033 [00:05<59:38,  1.76s/it]

  0%|          | 4/2033 [00:07<1:00:26,  1.79s/it]

  0%|          | 5/2033 [00:08<1:00:44,  1.80s/it]

  0%|          | 6/2033 [00:10<1:01:27,  1.82s/it]

  0%|          | 7/2033 [00:12<1:02:58,  1.87s/it]

  0%|          | 8/2033 [00:14<1:03:38,  1.89s/it]

  0%|          | 9/2033 [00:16<1:03:14,  1.87s/it]

  0%|          | 10/2033 [00:18<1:01:13,  1.82s/it]

  1%|          | 11/2033 [00:19<1:00:38,  1.80s/it]

  1%|          | 12/2033 [00:21<1:01:36,  1.83s/it]

  1%|          | 13/2033 [00:23<1:01:22,  1.82s/it]

  1%|          | 14/2033 [00:25<1:02:22,  1.85s/it]

  1%|          | 15/2033 [00:27<1:01:33,  1.83s/it]

  1%|          | 16/2033 [00:29<1:00:54,  1.81s/it]

  1%|          | 17/2033 [00:30<1:01:03,  1.82s/it]

  1%|          | 18/2033 [00:32<1:00:38,  1.81s/it]

  1%|          | 

Erro: [{'generated_text': 'musica'}]




  1%|          | 21/2033 [00:39<1:09:55,  2.09s/it]

  1%|          | 22/2033 [00:41<1:06:29,  1.98s/it]

  1%|          | 23/2033 [00:43<1:04:45,  1.93s/it]

  1%|          | 24/2033 [00:44<1:02:31,  1.87s/it]

  1%|          | 25/2033 [00:46<1:01:43,  1.84s/it]

  1%|▏         | 26/2033 [00:48<1:00:48,  1.82s/it]

  1%|▏         | 27/2033 [00:49<59:17,  1.77s/it]  

  1%|▏         | 28/2033 [00:51<58:07,  1.74s/it]

  1%|▏         | 29/2033 [00:53<56:30,  1.69s/it]

  1%|▏         | 30/2033 [00:54<56:35,  1.70s/it]

  2%|▏         | 31/2033 [00:56<58:13,  1.74s/it]

  2%|▏         | 32/2033 [00:58<59:41,  1.79s/it]

  2%|▏         | 33/2033 [01:00<59:02,  1.77s/it]

  2%|▏         | 34/2033 [01:02<59:38,  1.79s/it]

  2%|▏         | 35/2033 [01:03<58:15,  1.75s/it]

  2%|▏         | 36/2033 [01:05<57:33,  1.73s/it]

  2%|▏         | 37/2033 [01:07<58:02,  1.74s/it]

  2%|▏         | 38/2033 [01:09<57:32,  1.73s/it]

  2%|▏         | 39/2033 [01:10<58:19,  1.76s/it]

  2%|▏         


Model = ['google/flan-t5-large', False, None] || Acurácia: 0.17166748647319233
entrou


...

Erro: {'error': 'Model EleutherAI/gpt-j-6B is currently loading', 'estimated_time': 20.0}




 81%|████████  | 1651/2033 [26:19<1:24:51, 13.33s/it]

Erro: [{'generated_text': 'Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: coloca folk como favorito\nCenário: tocar/jogar\n\nExemplo 2\nComando: o alarme programado para as nove horas da noite de quinta-feira\nCenário: alarme\n\nExemplo 3\nComando: encontre-me as notícias mais populares do dia\nCenário: notícias\n\nExemplo 4\nComando: que reuniões estão disponíveis em março\nCenário: agenda\n\nExemplo 5\nComando: diminui volume\nCenário: áudio\n\nExemplo 6\nComando: em que dia vai estar ensolarado\nCenário: tempo\n\nExemplo 7\nComando: enumerar todos os eventos do meu calendário de março\nCenário: agenda\n\nExemplo 8\nComando: eu gosto de qualquer coisa do tiago nacarato\nCenário: musica\n\nExemplo 9\nComando: há um cozinheiro de chili este fim-de-semana\nCenário: agenda\n\nExemp



 81%|████████▏ | 1652/2033 [26:20<1:00:10,  9.48s/it]

 81%|████████▏ | 1653/2033 [26:20<42:55,  6.78s/it]  

 81%|████████▏ | 1654/2033 [26:21<31:13,  4.94s/it]

 81%|████████▏ | 1655/2033 [26:21<22:44,  3.61s/it]

 81%|████████▏ | 1656/2033 [26:22<16:47,  2.67s/it]

 82%|████████▏ | 1657/2033 [26:22<12:39,  2.02s/it]

 82%|████████▏ | 1658/2033 [26:23<09:47,  1.57s/it]

 82%|████████▏ | 1659/2033 [26:23<07:47,  1.25s/it]

 82%|████████▏ | 1660/2033 [26:24<06:22,  1.02s/it]

 82%|████████▏ | 1661/2033 [26:24<05:24,  1.15it/s]

 82%|████████▏ | 1662/2033 [26:25<04:43,  1.31it/s]

 82%|████████▏ | 1663/2033 [26:25<04:15,  1.45it/s]

 82%|████████▏ | 1664/2033 [26:26<03:52,  1.59it/s]

 82%|████████▏ | 1665/2033 [26:26<03:40,  1.67it/s]

 82%|████████▏ | 1666/2033 [26:27<03:29,  1.75it/s]

 82%|████████▏ | 1667/2033 [26:28<03:21,  1.82it/s]

 82%|████████▏ | 1668/2033 [26:28<03:16,  1.86it/s]

 82%|████████▏ | 1669/2033 [26:29<03:11,  1.91it/s]

 82%|████████▏ | 1670/2033 [26:29<03:06,


Model = ['EleutherAI/gpt-j-6B', False, None] || Acurácia: 0.08804722085587802
D

Acurácia prompt 1:  17.17%
{'iot': [('iot', 84), ('tempo', 18), ('culinaria', 16)], 'geral': [('iot', 83), ('tempo', 18), ('culinaria', 13), ('transporte', 3), ('perguntaserespostas', 2)], 'retirada': [('culinaria', 30), ('iot', 13), ('retirada', 1)], 'tocar/jogar': [('iot', 103), ('áudio', 65), ('musica', 57), ('tocar/jogar', 21), ('tempo', 6)], 'musica': [('iot', 25), ('musica', 23), ('áudio', 5), ('erro', 1), ('tempo', 1)], 'tempo': [('tempo', 94), ('iot', 28), ('culinaria', 2), ('perguntaserespostas', 2)], 'áudio': [('iot', 14), ('áudio', 11), ('musica', 7), ('tempo', 1), ('horaedata', 1)], 'horaedata': [('tempo', 57), ('horaedata', 15), ('alarme', 1)], 'alarme': [('tempo', 57), ('iot', 5), ('alarme', 2)], 'notícias': [('iot', 78), ('recomendaço', 1), ('notcias', 1), ('email', 1), ('geral', 1)], 'social': [('iot', 66), ('perguntaserespostas', 1), ('tempo', 1)], 'culinária': [('culinaria', 29), ('iot',

# Da Vinci 002 e 003

In [ ]:
prompts = [prob_prompt_scenarios(train_dataset, seed=55)]

print(prompts)

acc = []
pred = []
model_id_2 = [['text-davinci-002', False, None]]

for model in model_id_2:
  for prompt in prompts: 
    # print(prompt)
    accuracy, prediction = 0, 0

    accuracy, prediction = valida_davinci(model_id=model[0], dataset=valid_dataset, prompt=prompt, num_exemplos=-1)
    print(f'\nModel = {model} || Acurácia: {accuracy}')

    acc.append(accuracy)
    pred.append(prediction)

for i, p in enumerate(pred):
  print(prompt[0])
  print(f'\nAcurácia prompt {i+1}: {acc[i]*100: .2f}%')
  print(verifica_respostas(valid_dataset, p))


['Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: coloca folk como favorito\nCenário: tocar/jogar\n\nExemplo 2\nComando: o alarme programado para as nove horas da noite de quinta-feira\nCenário: alarme\n\nExemplo 3\nComando: encontre-me as notícias mais populares do dia\nCenário: notícias\n\nExemplo 4\nComando: que reuniões estão disponíveis em março\nCenário: agenda\n\nExemplo 5\nComando: diminui volume\nCenário: áudio\n\nExemplo 6\nComando: em que dia vai estar ensolarado\nCenário: tempo\n\nExemplo 7\nComando: enumerar todos os eventos do meu calendário de março\nCenário: agenda\n\nExemplo 8\nComando: eu gosto de qualquer coisa do tiago nacarato\nCenário: musica\n\nExemplo 9\nComando: há um cozinheiro de chili este fim-de-semana\nCenário: agenda\n\nExemplo 10\nComando: escrever 

100%|██████████| 2033/2033 [2:06:27<00:00,  3.73s/it]



Model = ['text-davinci-002', False, None] || Acurácia: 0.6517461878996557
D

Acurácia prompt 1:  65.17%
{'iot': [('iot', 78), ('IoT', 9), ('social', 8), ('IOT', 5), ('geral', 4)], 'geral': [('geral', 35), ('social', 25), ('perguntaserespostas', 15), ('listas', 12), ('iot', 5)], 'retirada': [('retirada', 23), ('culinária', 11), ('culinaria', 6), ('social', 2), ('culinária\n\nExemplo12\nComando:ConectaraodispositivoIoTparaacessaratemperaturadacasa\nCenário:IOT', 1)], 'tocar/jogar': [('tocar/jogar', 144), ('áudio', 41), ('social', 28), ('listas', 9), ('musica', 8)], 'musica': [('tocar/jogar', 24), ('musica', 13), ('listas', 5), ('social', 4), ('áudio', 3)], 'tempo': [('tempo', 119), ('social', 3), ('listas', 2), ('geral', 1), ('tempo\n\nExemplo12\nComando:qualopratododia\nCenário:culinária', 1)], 'áudio': [('áudio', 23), ('geral', 4), ('alarme', 3), ('\n\náudio', 2), ('social\n\nExemplo12\n\nComando:adicioneagrifecomoumcanaldenotícias\nCenário:notícias', 1)], 'horaedata': [('horaedata', 

In [ ]:
prompts = [prob_prompt_scenarios(train_dataset, seed=55)]

print(prompts)

acc = []
pred = []
model_id_3 = [['text-davinci-003', False, None]]

for model in model_id_3:
  for prompt in prompts: 
    # print(prompt)
    accuracy, prediction = 0, 0
    accuracy, prediction = valida_davinci(model_id=model[0], dataset=valid_dataset, prompt=prompt, num_exemplos=-1)
    print(f'\nModel = {model} || Acurácia: {accuracy}')

    acc.append(accuracy)
    pred.append(prediction)

for i, p in enumerate(pred):
  print(prompt[0])
  print(f'\nAcurácia prompt {i+1}: {acc[i]*100: .2f}%')
  print(verifica_respostas(valid_dataset, p))

['Defina o cenário para o último exemplo abaixo.\n\nCenários: social, transporte, agenda, tocar/jogar, notícias, horaedata, recomendação, email, iot, geral, áudio, listas, perguntaserespostas, culinaria, retirada, musica, alarme, tempo\n\nExemplo 1\nComando: coloca folk como favorito\nCenário: tocar/jogar\n\nExemplo 2\nComando: o alarme programado para as nove horas da noite de quinta-feira\nCenário: alarme\n\nExemplo 3\nComando: encontre-me as notícias mais populares do dia\nCenário: notícias\n\nExemplo 4\nComando: que reuniões estão disponíveis em março\nCenário: agenda\n\nExemplo 5\nComando: diminui volume\nCenário: áudio\n\nExemplo 6\nComando: em que dia vai estar ensolarado\nCenário: tempo\n\nExemplo 7\nComando: enumerar todos os eventos do meu calendário de março\nCenário: agenda\n\nExemplo 8\nComando: eu gosto de qualquer coisa do tiago nacarato\nCenário: musica\n\nExemplo 9\nComando: há um cozinheiro de chili este fim-de-semana\nCenário: agenda\n\nExemplo 10\nComando: escrever 

100%|██████████| 2033/2033 [3:53:51<00:00,  6.90s/it]



Model = ['text-davinci-003', False, None] || Acurácia: 0.6724053123462863
D

Acurácia prompt 1:  67.24%
{'iot': [('IoT', 56), ('IOT', 31), ('iot', 19), ('geral', 4), ('retirada', 3)], 'geral': [('perguntaserespostas', 36), ('geral', 25), ('social', 13), ('transporte', 10), ('notícias', 10)], 'retirada': [('retirada', 39), ('transporte', 2), ('culinaria', 1), ('geral', 1), ('culinária', 1)], 'tocar/jogar': [('tocar/jogar', 173), ('áudio', 60), ('música', 8), ('listas', 7), ('geral', 2)], 'musica': [('tocar/jogar', 33), ('música', 10), ('listas', 4), ('áudio', 4), ('email', 1)], 'tempo': [('tempo', 113), ('retirada', 6), ('transporte', 2), ('recomendação', 1), ('geral', 1)], 'áudio': [('áudio', 28), ('alarme', 6), ('perguntaserespostas', 1)], 'horaedata': [('horaedata', 37), ('dataehora', 17), ('data/hora', 8), ('hora/data', 6), ('horaedata.', 1)], 'alarme': [('alarme', 59), ('social', 1), ('horaedata', 1), ('Alarme', 1), ('tocar/jogar', 1)], 'notícias': [('notícias', 74), ('social', 2)